<a href="https://colab.research.google.com/github/KyleTy1er/Build-Week-Unit-2/blob/master/Build_Week_2_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install --upgrade category_encoders,

ERROR: Invalid requirement: 'category_encoders,'


In [0]:
pip install PDPbox

In [0]:
pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
import category_encoders as ce
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/KyleTy1er/Build-Week-Unit-2/master/fighters_df.csv', index_col=0)


In [0]:
pd.options.display.max_seq_items = None
pd.options.display.max_columns = 999

In [0]:
train, test = train_test_split(df)

In [0]:
drop_cols = ["fighter", "fighter_opponent", "age", "age_opponent", "age_ratio", "Referee", "date", "Reach_cms", "Reach_cms_opponent"]

target = "Stance"
features = df.columns.drop([target] + drop_cols)

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

In [0]:
X_train = X_train.fillna(X_train.mean())
y_train = y_train.fillna(method='ffill')
X_test = X_test.fillna(X_train.mean())
y_test = y_test.fillna(method='ffill')

In [0]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(7716, 214)
(7716,)
(2572, 214)
(2572,)


In [0]:
baseline_acc = train['Stance'].value_counts(normalize=True)[0].round(2)
print ("Majority Class - Orthodox:",baseline_acc)

Majority Class - Orthodox: 0.77


In [0]:
lr_pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    SimpleImputer(strategy="constant", fill_value="0"),
    LogisticRegression(max_iter=200),
)

lr_pipeline.fit(X_train, y_train)
# y_pred = lr_pipeline.predict(X_test)

# lr_score = accuracy_score(y_pred, y_test)*100
# improvement = lr_score - baseline_acc

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['location', 'Stance_opponent',
                                     'weight_class'],
                               drop_invariant=False, handle_missing='value',
                               handle_unknown='value', return_df=True,
                               use_cat_names=True, verbose=0)),
                ('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value='0',
                               missing_values=nan, strategy='constant',
                               verbose=0)),
                ('logisticregression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=200,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_

In [0]:
y_pred = lr_pipeline.predict(X_test)

In [0]:
lr_score = accuracy_score(y_pred, y_test)*100

In [0]:
improvement = lr_score - baseline_acc

In [0]:
print (lr_score)

75.69984447900467


In [0]:
import category_encoders as ce
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    SimpleImputer(strategy="constant", fill_value=0),
    RandomForestClassifier(n_jobs=-1, n_estimators=220),
)

# Fit on train, score on val
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

rfc_score = accuracy_score(y_pred, y_test)*100
# print('Test Accuracy', pipeline.score(X_train, y_test))
print (rfc_score)

77.52721617418351


In [0]:
from eli5.sklearn import PermutationImportance

transformer = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy="constant", fill_value=0),
)

X_train_transformed = transformer.fit_transform(X_train)
X_test_transformed = transformer.transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [0]:
rfc_model = RandomForestClassifier(
    n_jobs=-1,
    n_estimators=220,
)
rfc_model.fit(X_train_transformed, y_train)

permuter = PermutationImportance(
    rfc_model,
    scoring="accuracy",
    n_iter=5,
)

permuter.fit(X_test_transformed, y_test)

KeyboardInterrupt: ignored

In [0]:
from matplotlib import pyplot as plt

feature_importances = pd.Series(permuter.feature_importances_, features).sort_values()
feature_importances_std = pd.Series(permuter.feature_importances_std_, features).sort_values()

good_mask = feature_importances > 0
bad_mask = feature_importances < 0

fig, ax = plt.subplots(figsize=(8, 30))

feature_importances[good_mask].plot.barh(color="green", ecolor="#105010", xerr=feature_importances_std[good_mask], ax=ax)
ax.set_title("Positive Features")

fig.show()

In [0]:
fig, ax = plt.subplots(figsize=(8, 30))

feature_importances[bad_mask].plot.barh(color="red", ecolor="#501010", xerr=feature_importances_std[bad_mask], ax=ax)
ax.set_title("Negative Features")

fig.show()

In [0]:
X_test_transformed = pd.DataFrame(X_test_transformed)

In [0]:
X_test_transformed

In [0]:
X_train

In [0]:
from pdpbox.pdp import pdp_isolate, pdp_plot

name = "Weight_lbs"

isolated = pdp_isolate(
        model=model,
        dataset=X_test_transformed,
        model_features=X_test.columns,
        feature=name,
        num_grid_points=50,
)

fig, ax = pdp_plot(isolated, feature_name=name)
fig.show()

In [0]:
# Get feature importances
rf = pipeline.named_steps['randomforestclassifier']
importances = pd.Series(rf.feature_importances_, X_train.columns)

# Plot feature importances
%matplotlib inline
import matplotlib.pyplot as plt

n = 20
plt.figure(figsize=(10,n/2))
plt.title(f'Top {n} features')
importances.sort_values()[-n:].plot.barh(color='grey');

In [0]:
# # The status_group column is the target
# target = 'status_group'

# # Get a dataframe with all train columns except the target & id
# train_features = train.drop(columns=[target, 'id'])

# # Get a list of the numeric features
# numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# # Get a series with the cardinality of the nonnumeric features
# cardinality = train_features.select_dtypes(exclude='number').nunique()

# # Get a list of all categorical features with cardinality <= 50
# categorical_features = cardinality[cardinality <= 50].index.tolist()

# # Combine the lists 
# features = numeric_features + categorical_features
# print(features)

In [0]:
# # Ignore warnings

# transformers = make_pipeline(
#     ce.OrdinalEncoder(), 
#     SimpleImputer(strategy='median') 
# )

# X_train_transformed = transformers.fit_transform(X_train)
# X_val_transformed = transformers.transform(X_val)

# model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
# model.fit(X_train_transformed, y_train)

In [0]:
import eli5
from eli5.sklearn import PermutationImportance

permuter = PermutationImportance(
    model,
    scoring='accuracy',
    n_iter=5,
    random_state=42
)

permuter.fit(X_val_transformed, y_val)

In [0]:
# feature_names = X_val.columns.tolist()
# pd.Series(permuter.feature_importances_, feature_names).sort_values()

In [0]:
# eli5.show_weights(
#     permuter,
#     top=None,
#     feature_names=feature_names